In [ ]:
%pip install -r ../requirements.txt

In [1]:
# Only run once do not touch again
base_dir: bool = False

In [2]:
import os, sys

if not base_dir:
    os.chdir('../')
    base_dir = True

import tensorflow as tf
from tensorflow import keras

import numpy as np

import cv2

In [3]:
np.__version__, tf.__version__, keras.__version__

('2.0.2', '2.18.0', '3.7.0')

In [4]:
model = keras.models.load_model('models/SmallCNN.keras')

c:\Users\Theon\Downloads\view-prediction\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [40]:
from utils.data_tools import inbreast, cbis_ddsm

cbis_image_batch_gen = cbis_ddsm.image_batch(compress=True, image_scale='Normalize', to_rgb=True)
inbreast_image_batch_gen = inbreast.image_batch(compress=True, image_scale='Normalize', to_rgb=True)

In [29]:
WRONG_L_MLO = []
WRONG_L_CC = []
WRONG_R_MLO = []
WRONG_R_CC = []

In [43]:
for generator in [inbreast_image_batch_gen]:    
    for batch in generator:

        images = np.array(batch[1])

        y_pred = model.predict(images)

        y_pred_binary = (y_pred >= 0.5).astype(int)

        for pred, data, image in zip(y_pred_binary, *batch):

            lat, view = data['Laterality'], data['View']

            y_true = 1 if view == 'MLO' else 0

            if y_true != pred[0]:

                print(y_true, pred)

                if lat == 'L':

                    if view == 'CC':

                        WRONG_L_CC.append(image)

                    else:
                
                        WRONG_L_MLO.append(image)

                else:


                    if view == 'CC':

                        WRONG_R_CC.append(image)

                    else:
                
                        WRONG_R_MLO.append(image)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1 [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1 [0]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1 [0]
0 [1]


In [48]:
list(map(lambda arr: arr.__len__(), [WRONG_L_CC, WRONG_L_MLO, WRONG_R_CC, WRONG_R_MLO]))

[7, 8, 7, 13]

In [47]:
WRONG_R_CC = WRONG_R_CC + [np.zeros(shape=(224,224,3))] * 4

In [49]:
num_display_examples: int = 7

all_mammographs_ = [WRONG_L_CC, WRONG_L_MLO, WRONG_R_CC, WRONG_R_MLO]

assert all(map(lambda arr: arr.__len__() >= num_display_examples, all_mammographs_)), f"Not all images have {num_display_examples} examples"

In [51]:
from utils.visualizer import visualizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

display_image_size = (100, 100)

def mammograph_labels_gen(): 
    yield from ['L_CC', 'L_MLO', 'R_CC', 'R_MLO']

mammograph_labels_ = mammograph_labels_gen()

fig, axes = plt.subplots(4, num_display_examples, figsize=display_image_size)

def ax_gen():
    yield from axes.flat

ax_ = ax_gen()

for column, mammographs in zip(range(4), all_mammographs_):

    for i in range(num_display_examples):

        new_plot = visualizer.grad_cam(mammographs[i], model, color='orange')

        # Get model prediction

        prediction = 'MLO' if model.predict(
            np.expand_dims(mammographs[i], axis=0), verbose = 0
        ) >= 0.5 else 'CC'

        ax = next(ax_)

        if i == 0:

            curr_label = next(mammograph_labels_)

            ax.set_ylabel(curr_label, labelpad=10, fontsize=display_image_size[0] / 2, color='blue')

        # Resized to save memory
        ax.imshow(cv2.resize(new_plot._A.data, display_image_size))

        ax.set_xlabel(prediction, labelpad=10, fontsize=display_image_size[0] / 2, color='green' if curr_label.split('_')[1] ==  prediction else 'red')
        
        # ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

        clear_output()

plt.show()